In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import spatial
from sklearn import metrics
import math
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torchvision.utils import make_grid
import torch.nn.functional as F
from torch import optim
from tqdm import tqdm

from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm
import cupy
from cuml.feature_extraction.text import TfidfVectorizer
from cuml.neighbors import NearestNeighbors
from cuml import metrics
import cudf
import torch

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import os

import warnings
warnings.filterwarnings('ignore')

timm_path = "../input/timm-pytorch-image-models/pytorch-image-models-master"
import sys
sys.path.append(timm_path)
import timm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [2]:
cv = True
test = pd.read_csv("../input/shopee-product-matching/test.csv")
if(len(test)>3):
    cv = False
else:
    test  = pd.read_csv("../input/shopee-product-matching/train.csv")
    test = test.iloc[0:350,:]
test.head()

,posting_id,image,image_phash,title,label_group
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069


In [3]:
import os
if (cv == True):
    img_path = '../input/shopee-product-matching/train_images'
else:
    img_path = "../input/shopee-product-matching/test_images"
paths = [os.path.join(img_path,x) for x in test.image]
test['path'] = paths
test.head()

,posting_id,image,image_phash,title,label_group,path
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,../input/shopee-product-matching/train_images/...
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045,../input/shopee-product-matching/train_images/...
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,../input/shopee-product-matching/train_images/...
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188,../input/shopee-product-matching/train_images/...
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069,../input/shopee-product-matching/train_images/...


In [4]:
tmp = test.groupby('image_phash').posting_id.agg('unique').to_dict()
test['preds4'] = test.image_phash.map(tmp)
test.head()

,posting_id,image,image_phash,title,label_group,path,preds4
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,../input/shopee-product-matching/train_images/...,[train_129225211]
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045,../input/shopee-product-matching/train_images/...,[train_3386243561]
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,../input/shopee-product-matching/train_images/...,[train_2288590299]
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188,../input/shopee-product-matching/train_images/...,[train_2406599165]
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069,../input/shopee-product-matching/train_images/...,[train_3369186413]


In [5]:
class GeM2(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM2,self).__init__()
        self.p = nn.Parameter(torch.ones(1)*p)
        self.eps = eps

    def forward(self, x):
        return self.gem(x, p=self.p, eps=self.eps)
        
    def gem(self, x, p=3, eps=1e-6):
        return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)
        
    def __repr__(self):
        return self.__class__.__name__ + '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + ', ' + 'eps=' + str(self.eps) + ')'
class ArcModule(nn.Module):
    def __init__(self, in_features, out_features, s = 10, m = 0.5):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_normal_(self.weight)

        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = torch.tensor(math.cos(math.pi - m))
        self.mm = torch.tensor(math.sin(math.pi - m) * m)
        
      

    def forward(self, inputs, labels=None):
        cos_th = F.linear(inputs, F.normalize(self.weight))
        cos_th = cos_th.clamp(-1, 1) 
        sin_th = torch.sqrt(1.0 - torch.pow(cos_th, 2)+ 1e-7)
        cos_th_m = cos_th * self.cos_m - sin_th * self.sin_m
        # print(type(cos_th), type(self.th), type(cos_th_m), type(self.mm))
        cos_th_m = torch.where(cos_th > self.th, cos_th_m, cos_th - self.mm)

        cond_v = cos_th - self.th
        cond = cond_v <= 0
        cos_th_m[cond] = (cos_th - self.mm)[cond]

        if labels.dim() == 1:
            labels = labels.unsqueeze(-1)
        onehot = torch.zeros(cos_th.size()).to(device)
        labels = labels.type(torch.LongTensor).to(device)
        onehot.scatter_(1, labels, 1.0)
        outputs = onehot * cos_th_m + (1.0 - onehot) * cos_th
        outputs = outputs * self.s
        return outputs

In [6]:
class Model1(nn.Module):
    def __init__(self,network,output_size = 11014 ,pretrained=False):
        super().__init__()
        self.op = output_size
        self.network = network
        self.backbone = timm.create_model(self.network , features_only=True, pretrained=pretrained)
        self.gem2 = GeM2()
        self.fc1 = nn.Linear(2048,512)
        self.do = nn.Dropout(p=0.25)
        self.bn1 = nn.BatchNorm1d(2048)
        self.bn2 = nn.BatchNorm1d(512)
        self.pl = nn.PReLU()
        self.margin = ArcModule(in_features=512, out_features = self.op)
        
        nn.init.kaiming_normal_(self.fc1.weight)
        nn.init.zeros_(self.fc1.bias)

    def forward(self,x,labels=None):
        y = self.backbone(x)
        #y1 = self.gem1(y[2])
        y2 = self.gem2(y[4])
        
        #y1 = y1.view(x.shape[0],-1)
        y2 = y2.view(x.shape[0],-1)
        y2 = self.bn1(y2)
        
        #concat = torch.cat((y1,y2),dim = 1)
        fc1 = self.do(self.fc1(y2))
        feat1 = self.bn2(fc1)
        feat2 =  F.normalize(feat1)
        if labels is not None:
            return self.margin(feat2, labels)
        else:
            return feat2


In [7]:
train_aug = A.Compose(
    [  

        A.Resize(512,512,p=1.0),
     A.Normalize(p=1.0),
        ToTensorV2(p=1.0)
    ]
)

In [8]:
class Shop(Dataset):
    def __init__(self,df,augs=None):
        self.df = df
        self.augs = augs
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        img_src = self.df.path.iloc[idx]
        image = cv2.imread(img_src)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.uint8)

        if (self.augs):
            transformed = self.augs(image=image)
            image = transformed['image']
        
        return image

In [9]:
da = Shop(test,augs=train_aug)
loader =  DataLoader(da,shuffle=False,
                        num_workers=4,
                        batch_size=16,
                        pin_memory=True)


def generate_test_features(test_loader,M, path):
    mod =  M
    WEIGHTS_FILE = path
    mod.load_state_dict(torch.load(WEIGHTS_FILE))
    mod.to(device)
    mod.eval()
    bar = tqdm(test_loader)
    
    FEAS = []

    with torch.no_grad():
        for batch_idx, (images) in enumerate(bar):

            images = images.to(device)

            features = mod(images)

            FEAS += [features.detach().cpu()]

    FEAS = torch.cat(FEAS).cpu().numpy()
    
    del mod
    
    return FEAS

In [10]:
FEAS1 = generate_test_features(loader,Model1('seresnet50'), "../input/serex-123/imageT123_seresnext.pth")
FEAS2 =generate_test_features(loader,Model1('res2next50') ,"../input/re2net-even/image_modelTeven_512_res2.pth")

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

In [11]:
FEASY = (FEAS1 + FEAS2)/2
del FEAS1,FEAS2

In [12]:
class Rare_Model(nn.Module):
    def __init__(self,output_size =10 ,pretrained=False):
        super().__init__()
        self.op = output_size
        self.backbone = timm.create_model('tf_efficientnet_b2_ns', features_only=True, 
                                          pretrained=pretrained)
        self.gem2 = GeM2()
        self.fc1 = nn.Linear(352,144)
        self.fc2 = nn.Linear(144,self.op)
        self.do = nn.Dropout(p=0.25)
        self.bn1 = nn.BatchNorm1d(352)
        self.bn2 = nn.BatchNorm1d(144)
        self.pl = nn.PReLU()
        
        nn.init.kaiming_normal_(self.fc1.weight)
        nn.init.zeros_(self.fc1.bias)

    def forward(self,x,labels=None):
        y = self.backbone(x)
        #y1 = self.gem1(y[2])
        y2 = self.gem2(y[4])
        
        #y1 = y1.view(x.shape[0],-1)
        y2 = y2.view(x.shape[0],-1)
        y2 = self.bn1(y2)
        
        #concat = torch.cat((y1,y2),dim = 1)
        fc1 = self.do(self.fc1(y2))
        feat1 = self.pl(self.bn2(fc1))
        
        feat1 = self.fc2(feat1)
        
        return feat1

In [13]:
def generate_rarity(test_loader,M, path):
    mod =  M
    WEIGHTS_FILE = path
    mod.load_state_dict(torch.load(WEIGHTS_FILE))
    mod.to(device)
    mod.eval()
    bar = tqdm(test_loader)
    
    FEAS = []

    with torch.no_grad():
        for batch_idx, (images) in enumerate(bar):

            images = images.to(device)

            output = mod(images)
            output = output.softmax(1)
            outputs = torch.argmax(output, dim=1)

            FEAS += [outputs.detach().cpu()]

    FEAS = torch.cat(FEAS).cpu().numpy()
    
    del mod
    
    return FEAS

In [14]:
FEASR = generate_rarity(loader,Rare_Model(), "../input/last-exp/Fold 0 model with val_acc 0.8086131386861314.pth")

  0%|          | 0/22 [00:00<?, ?it/s]

In [15]:
def thres_func(x):
    if (x<1):
        return 0.69
    elif (1<=x<6):
        return 0.8
    else:
        return 0.85
    

In [16]:
vfunc = np.vectorize(thres_func)

In [17]:
th = vfunc(FEASR)
del FEASR

In [18]:
FEASY =  np.concatenate((FEASY,np.expand_dims(th, axis=1)),axis=1)

In [19]:
import gc
def similar(FEAS):
    preds = []
    CHUNK = 900
#te = ar[:,1:]
    print('Finding similar titles...')
    CTS = len(test)//CHUNK
    if len(test)%CHUNK!=0: CTS += 1
    for j in range( CTS ):
        a = j*CHUNK
        b = (j+1)*CHUNK
        b = min(b,len(test))
        print('chunk',a,'to',b)
        cts = metrics.pairwise_distances(FEAS[:,:-1][a:b],FEAS[:,:-1], metric='cosine')
        cts = 1-cts
        for k in range(b-a):
            IDX = np.where(cts[k,]>FEAS[a:b][:,-1][k])[0]
            o = test.iloc[cupy.asnumpy(IDX)].posting_id.values
            preds.append(o)
            
   
    
    return preds

#del te,dx,modelx
_ = gc.collect()

In [20]:

test['preds1'] = similar(FEASY)
del FEASY

Finding similar titles...
chunk 0 to 350


In [21]:
torch.cuda.empty_cache()

In [22]:
df_cu = cudf.DataFrame(test)
if (cv == True):
    odel = TfidfVectorizer(stop_words = 'english', binary = True, max_features = 250)
    text_embeddings = odel.fit_transform(df_cu.title).toarray()
    print('text embeddings shape',text_embeddings.shape)
else:
    odel = TfidfVectorizer(stop_words = 'english', binary = True, max_features = 25_000)
    text_embeddings = odel.fit_transform(df_cu.title).toarray()
    print('text embeddings shape',text_embeddings.shape)

text_embeddings =  cupy.concatenate((text_embeddings,cupy.expand_dims(cupy.array(th), axis=1)),axis=1)
print(text_embeddings.shape)
predsx = []
CHUNK = 1024

print('Finding similar titles...')
CTS = len(test)//CHUNK
if len(test)%CHUNK!=0: CTS += 1
for j in range( CTS ):
    a = j*CHUNK
    b = (j+1)*CHUNK
    b = min(b,len(test))
    print('chunk',a,'to',b)

        # COSINE SIMILARITY DISTANCE
    cts = cupy.matmul( text_embeddings[:,:-1], text_embeddings[:,:-1][a:b].T).T

    for k in range(b-a):
        IDX = cupy.where(cts[k,]>text_embeddings[a:b][:,-1][k])[0]
        o = test.iloc[cupy.asnumpy(IDX)].posting_id.values
        predsx.append(o)
    
del odel,text_embeddings,df_cu
_ = gc.collect()

text embeddings shape (350, 250)
(350, 251)
Finding similar titles...
chunk 0 to 350


In [23]:
test['preds2'] =  predsx
del predsx

In [24]:
_ = gc.collect()

In [25]:
torch.cuda.empty_cache()

In [26]:
def getMetric(col):
    def f1score(row):
        n = len( np.intersect1d(row.target,row[col]) )
        return 2*n / (len(row.target)+len(row[col]))
    return f1score
def combine_for_sub(row):
    x = np.concatenate([row.preds2,row.preds1,row.preds4])
    return ' '.join( np.unique(x) )

def combine_for_cv(row):
    x = np.concatenate([row.preds2,row.preds1,row.preds4])
    return np.unique(x)
tmp = test.groupby('label_group').posting_id.agg('unique').to_dict()
test['target'] = test.label_group.map(tmp)
test['oof'] = test.apply(combine_for_cv,axis=1)
test['f1'] = test.apply(getMetric('oof'),axis=1)
print('CV Score =', test.f1.mean() )

CV Score = 0.9154285714285714


In [27]:
test['matches'] = test.apply(combine_for_sub,axis=1)
test.head()

,posting_id,image,image_phash,title,label_group,path,preds4,preds1,preds2,target,oof,f1,matches
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,../input/shopee-product-matching/train_images/...,[train_129225211],[train_129225211],"[train_129225211, train_2051205225]",[train_129225211],"[train_129225211, train_2051205225]",0.666667,train_129225211 train_2051205225
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045,../input/shopee-product-matching/train_images/...,[train_3386243561],"[train_3386243561, train_1816968361]",[train_3386243561],[train_3386243561],"[train_1816968361, train_3386243561]",0.666667,train_1816968361 train_3386243561
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,../input/shopee-product-matching/train_images/...,[train_2288590299],[train_2288590299],"[train_2288590299, train_3025597522]",[train_2288590299],"[train_2288590299, train_3025597522]",0.666667,train_2288590299 train_3025597522
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188,../input/shopee-product-matching/train_images/...,[train_2406599165],[train_2406599165],[train_2406599165],[train_2406599165],[train_2406599165],1.000000,train_2406599165
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069,../input/shopee-product-matching/train_images/...,[train_3369186413],[train_3369186413],[],[train_3369186413],[train_3369186413],1.000000,train_3369186413


In [28]:
test[['posting_id','matches']].to_csv('submission.csv',index=False)
sub = pd.read_csv('submission.csv')
sub.head()

,posting_id,matches
0,train_129225211,train_129225211 train_2051205225
1,train_3386243561,train_1816968361 train_3386243561
2,train_2288590299,train_2288590299 train_3025597522
3,train_2406599165,train_2406599165
4,train_3369186413,train_3369186413


In [29]:
test.to_csv('test.csv',index=False)